In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("SparkHelloWorldStream")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc = spark.sparkContext

22/05/24 09:50:34 WARN Utils: Your hostname, Hocines-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.164 instead (on interface en0)
22/05/24 09:50:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/24 09:50:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Structured Streaming 
# based on Data Frame/ internally there is RDD
# Micro Batch, trigger is default one, as soon as possible when data arrives
# trigger time approx 100 ms to 1 sec
# spark batch programming, we use spark.read
# in streaming, we use readStream
# socket is predefined word, represent data comes from TCP/IP socket
# spark is client
lineStreamDf = spark\
                .readStream\
                .format("socket")\
                .option("host", "localhost")\
                .option("port", 9999)\
                .load()

# now spark shall connect the server running on port 9999
# then it will wait the input from server
# user/developer can write something on server console and Press Enter key
# to send to spark / client


22/05/24 10:25:55 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [4]:
# printSchema vary based on driver we use. for socket, we have only value
lineStreamDf.printSchema()
# show will not work for stream, it will cause error
#lineStreamDf.show() # good for batchs, not for stream

root
 |-- value: string (nullable = true)



In [5]:
# print data on console 
# for stream write, we should use writeStream
# for batch use write
# append is mode
# console is keyword, present console output
# writeStream is an action, which runs the whole show,
# reading data, trigger window etc by writeStream
# in general 1 sec trigger used if not mentioned, default
echoOnconsole = lineStreamDf\
                .writeStream\
                .outputMode("append")\
                .format("console")\
                .start()

# start basically start the streaming, listen for input
# as input arrives, it starts processing
# listen for jupyter console for the output

22/05/24 10:32:06 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/y4/h7s1_2k959z8nfw5xqv11nmr0000gn/T/temporary-cf5f01ed-36e5-4833-8428-f906b14e53a8. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/05/24 10:32:06 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-------+
|  value|
+-------+
|  hello|
|welcome|
+-------+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|welcome to spark ...|
+--------------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+------+
| value|
+------+
|hocine|
+------+

-------------------------------------------
Batch: 4
-------------------------------------------
+--------+
|   value|
+--------+
|makhlouf|
+--------+

-------------------------------------------
Batch: 5
-------------------------------------------
+------+
| value|
+------+
|dz2022|
+------+

-------------------------------------------
Batch: 6
-------